<a href="https://colab.research.google.com/github/wessware/credit_worth_evaluation_ML_modelling/blob/main/models/shipped_model_credit_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
loan_data_test = pd.read_csv('/content/sample_data/bankrecords.csv')

In [3]:
del loan_data_test['Loan_ID']

In [4]:
x_ins = loan_data_test.drop(columns =['Loan_Status'])
y_ins = loan_data_test['Loan_Status']

In [5]:
y = pd.DataFrame(data=y_ins)

In [7]:
y_num = {'Loan_Status': {'Y' : 1, 'N' : 0}}

y = y.replace(y_num)

In [8]:
numerical_cols = x_ins.select_dtypes(include=np.number).columns.tolist()
categorical_cols = x_ins.select_dtypes('object').columns.tolist()

In [9]:
impute_ins = SimpleImputer(strategy='mean').fit(x_ins[numerical_cols])
impute_ins.transform(x_ins[numerical_cols])

array([[0.00000000e+00, 5.84900000e+03, 0.00000000e+00, 1.46412162e+02,
        3.60000000e+02, 1.00000000e+00],
       [1.00000000e+00, 4.58300000e+03, 1.50800000e+03, 1.28000000e+02,
        3.60000000e+02, 1.00000000e+00],
       [0.00000000e+00, 3.00000000e+03, 0.00000000e+00, 6.60000000e+01,
        3.60000000e+02, 1.00000000e+00],
       ...,
       [1.00000000e+00, 8.07200000e+03, 2.40000000e+02, 2.53000000e+02,
        3.60000000e+02, 1.00000000e+00],
       [2.00000000e+00, 7.58300000e+03, 0.00000000e+00, 1.87000000e+02,
        3.60000000e+02, 1.00000000e+00],
       [0.00000000e+00, 4.58300000e+03, 0.00000000e+00, 1.33000000e+02,
        3.60000000e+02, 0.00000000e+00]])

In [11]:
encode_ins = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(x_ins[categorical_cols])
cols_encode_ins = list(encode_ins.get_feature_names_out(categorical_cols))
x_ins[cols_encode_ins] = encode_ins.transform(x_ins[categorical_cols])

In [12]:
loan_data_test_inst = x_ins[numerical_cols + cols_encode_ins]

In [18]:
loan_missing_1 = loan_data_test_inst.LoanAmount.mean()
loan_data_test_inst.LoanAmount = loan_data_test_inst.LoanAmount.fillna(loan_missing_1)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [19]:
loan_missing_2 = loan_data_test_inst.Loan_Amount_Term.median()
loan_data_test_inst.Loan_Amount_Term = loan_data_test_inst.Loan_Amount_Term.fillna(loan_missing_2)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
loan_missing_3 = loan_data_test_inst.Dependents.median()
loan_data_test_inst.Dependents = loan_data_test_inst.Dependents.fillna(loan_missing_3)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
loan_missing_4 = loan_data_test_inst.Credit_History.median()
loan_data_test_inst.Credit_History = loan_data_test_inst.Credit_History.fillna(loan_missing_4)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [22]:
loan_data_test_inst.isnull().sum()

Dependents                 0
ApplicantIncome            0
CoapplicantIncome          0
LoanAmount                 0
Loan_Amount_Term           0
Credit_History             0
Gender_Female              0
Gender_Male                0
Gender_nan                 0
Married_No                 0
Married_Yes                0
Married_nan                0
Education_Graduate         0
Education_Not Graduate     0
Self_Employed_No           0
Self_Employed_Yes          0
Self_Employed_nan          0
Property_Area_Rural        0
Property_Area_Semiurban    0
Property_Area_Urban        0
dtype: int64

In [23]:
x = loan_data_test_inst

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=30)

In [25]:
scale_ins = StandardScaler()

x_train_scaled = scale_ins.fit_transform(x_train)
x_test_scaled = scale_ins.fit_transform(x_test)

In [26]:
svm = SVC(kernel= 'linear', C=1, random_state=1, probability=True)
svm.fit(x_train_scaled, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1, kernel='linear', probability=True, random_state=1)

In [27]:
y_pred = svm.predict(x_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy = {:.1f}%'.format(accuracy*100))

Accuracy = 80.0%


In [28]:
mean_squared_error(y_test, y_pred)
math.sqrt(mean_squared_error(y_test, y_pred))

0.4472135954999579

In [29]:
def predict_instance(svm, single_input):
    input_ins = pd.DataFrame([single_input])
    
    input_ins[numerical_cols] = impute_ins.transform(input_ins[numerical_cols])
    #input_ins[numerical_cols] = scale_ins.transform(input_ins[numerical_cols])
    input_ins[cols_encode_ins] = encode_ins.transform(input_ins[categorical_cols])
    
    x_ins_input = input_ins[numerical_cols + cols_encode_ins]
    
    predictions = svm.predict(x_ins_input)[0]
    probability = svm.predict_proba(x_ins_input)[0][list(svm.classes_).index(predictions)]
    
    return predictions, probability

In [32]:
single_instance_0 = {
    'Gender' : 'Male', 
    'Married' : 'Yes',
    'Dependents' : 5,
    'Education' :  'Graduate', 
    'Self_Employed' : 'Yes', 
    'ApplicantIncome' : 30000, 
    'CoapplicantIncome' : 24000, 
    'LoanAmount' : 6000, 
    'Loan_Amount_Term' : 360, 
    'Credit_History' : 1, 
    'Property_Area' : 'Urban', 
    'Loan_Status' : 'Y'
}

In [30]:
single_instance_1 = {
    'Gender' : 'Male', 
    'Married' : 'No',
    'Dependents' : 9,
    'Education' :  'Not Graduate', 
    'Self_Employed' : 'Yes', 
    'ApplicantIncome' : 3000, 
    'CoapplicantIncome' : 200, 
    'LoanAmount' : 60000, 
    'Loan_Amount_Term' : 360, 
    'Credit_History' : 0, 
    'Property_Area' : 'Rural', 
    'Loan_Status' : 'N'
}

In [33]:
output_variable_0 = predict_instance(svm, single_instance_0)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [34]:
output_variable_0 = predict_instance(svm, single_instance_0)
output_variable_0

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


(0, 0.9999998999999902)

In [35]:
output_variable_1 = predict_instance(svm, single_instance_1)
output_variable_1

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


(1, 0.9594288789827102)